In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.00001
IOU_THR = 0.7
PREDICTION_PATH = "dataset/predictions/09-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
29282,ID_XDDvmd.jpg,cssvd,0.000107,3142.875000,0.000000,3264.000000,1286.156250
8934,ID_VqylTD.jpeg,healthy,0.000140,68.125000,0.000000,236.875000,102.187500
241628,ID_tf5cZg.jpeg,healthy,0.000249,0.000000,0.000000,299.742157,314.999969
102495,ID_GRoUWA.JPG,healthy,0.013023,617.976562,14.343750,1153.875000,615.187500
183511,ID_TNiZUE.jpg,cssvd,0.000043,0.131836,387.070312,45.120850,810.000000


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,healthy,0.698242,72.265617,0.000000,3810.546631,1687.499878,2
1,ID_cWEAQI.jpeg,anthracnose,0.010094,2726.562256,638.671814,3874.999756,1568.359253,0
2,ID_cWEAQI.jpeg,healthy,0.006073,166.015610,0.000000,1314.453003,544.921814,2
3,ID_cWEAQI.jpeg,anthracnose,0.004757,3523.437256,7.812500,3999.999756,753.906189,0
4,ID_cWEAQI.jpeg,healthy,0.002958,366.210907,1406.249878,1158.203003,1800.000000,2


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
66888,ID_S1PHIl.jpg,healthy,0.00090,948.164062,406.054688,1038.867188,486.210938,2
163467,ID_ZQ6oNQ.jpeg,healthy,0.00014,3130.312256,1846.687378,4028.062256,2555.437256,2


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
# predictions["ID_CGnAYP.jpg"]

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
# for k, v in predictions.items():
#     r = filtered_predictions[k]
#     if len(v["boxes"]) != len(r["boxes"]):
#         print(k, len(v["boxes"]), len(r["boxes"]))
#         print("Boxes before NMS:", v["boxes"])
#         print("Boxes after NMS:", r["boxes"])
#         print("Scores before NMS:", v["scores"])
#         print("Scores after NMS:", r["scores"])
#         print("Labels before NMS:", v["labels"])
#         print("Labels after NMS:", r["labels"])
#         print()
#         break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.010094,2726.562256,638.671814,3874.999756,1568.359253,0
1,ID_cWEAQI.jpeg,anthracnose,0.004757,3523.437256,7.812500,3999.999756,753.906189,0
2,ID_cWEAQI.jpeg,anthracnose,0.001561,3316.406006,1.953125,4000.000000,1019.531189,0
3,ID_cWEAQI.jpeg,anthracnose,0.001536,3757.812256,1.953125,3999.999756,833.984314,0
4,ID_cWEAQI.jpeg,anthracnose,0.001519,2949.218506,692.382751,3999.999756,1488.281128,0


In [14]:
len(filtered_df), len(test_df)

(241830, 243900)

In [15]:
filtered_df["class"].value_counts()

class
healthy        92990
cssvd          84383
anthracnose    64457
Name: count, dtype: int64

In [16]:
filtered_df["class"].value_counts(normalize=True)

class
healthy        0.384526
cssvd          0.348935
anthracnose    0.266538
Name: proportion, dtype: float64

In [17]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [18]:
test_df: pd.DataFrame = filtered_df.copy()

In [19]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.01009368896484375,
  'ymin': 2726.562255859375,
  'xmin': 638.6718139648438,
  'ymax': 3874.999755859375,
  'xmax': 1568.3592529296875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.004756927490234375,
  'ymin': 3523.437255859375,
  'xmin': 7.812499523162842,
  'ymax': 3999.999755859375,
  'xmax': 753.9061889648438,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0015611648559570312,
  'ymin': 3316.406005859375,
  'xmin': 1.9531248807907104,
  'ymax': 4000.0,
  'xmax': 1019.5311889648438,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0015363693237304688,
  'ymin': 3757.812255859375,
  'xmin': 1.9531248807907104,
  'ymax': 3999.999755859375,
  'xmax': 833.9843139648438,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0015192031

In [20]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [21]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [22]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
75430,ID_MUYBb1.jpg,healthy,0.001681,166.250000,348.125000,580.000000,620.000000,2
124905,ID_VQKJAq.jpg,cssvd,0.394287,1524.000000,168.750000,2032.000000,1021.000000,1
192385,ID_onW3aY.jpg,healthy,0.000203,0.032959,165.058594,48.153076,312.714844,2
32536,ID_kLKJFt.jpg,healthy,0.000142,0.000000,218.320312,35.299072,410.273438,2
47090,ID_PzpVJN.jpg,cssvd,0.000223,6.375000,1848.750000,986.531250,2448.000000,1
4183,ID_bu0b88.jpg,anthracnose,0.000180,2314.125000,758.625000,2856.000000,1673.437500,0
126423,ID_FbsP3U.JPG,anthracnose,0.000351,585.703125,342.656250,2690.250000,1000.875000,0
126985,ID_msJc8y.jpg,cssvd,0.001143,0.000000,37.453125,680.929688,756.234375,1
97619,ID_vfelPl.jpg,healthy,0.000070,5.976562,1778.625000,427.523438,2448.000000,2
233741,ID_S4UOLv.jpg,healthy,0.000287,71.703117,0.025391,206.781235,48.369137,2


In [23]:
test_df["class"].value_counts()

class
healthy        92990
cssvd          84383
anthracnose    64457
Name: count, dtype: int64

In [24]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [25]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/09-submissions.csv'